# Create OH slabs
---

# Import Modules

In [1]:
import os
print(os.getcwd())
import sys
import time; ti = time.time()

from IPython.display import display

import random
import numpy as np
from numpy import dot
import pandas as pd
# pd.set_option("display.max_columns", None)
# pd.options.display.max_colwidth = 20

from ase import Atoms
from ase import io

# #########################################################
from proj_data import metal_atom_symbol

# #########################################################
from methods import get_df_coord
from methods import (
get_df_jobs_data,
    get_df_jobs_anal,
    get_df_active_sites,
    get_df_atoms_sorted_ind,
    )

# #########################################################
from local_methods import get_neighbor_metal_atom
from local_methods import get_ads_pos_oh
from local_methods import M

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/dft_workflow/job_analysis/create_oh_slabs


In [2]:
from methods import isnotebook    
isnotebook_i = isnotebook()
if isnotebook_i:
    from tqdm.notebook import tqdm
    verbose = True
else:
    from tqdm import tqdm
    verbose = False

### Read data objects with methods

In [3]:
df_jobs_data = get_df_jobs_data(exclude_wsl_paths=True)

df_jobs_anal = get_df_jobs_anal()

df_active_sites = get_df_active_sites()

df_atoms_sorted_ind = get_df_atoms_sorted_ind()

### Filtering down to only `oer_adsorbate` jobs

In [4]:
df_ind = df_jobs_anal.index.to_frame()

df_jobs_anal = df_jobs_anal.loc[
    df_ind[df_ind.job_type == "oer_adsorbate"].index
    ]

In [5]:
df_jobs_anal_i = df_jobs_anal[df_jobs_anal.job_completely_done == True]

var = "o"
df_jobs_anal_i = df_jobs_anal_i.query('ads == @var')

var = "NaN"
df_jobs_anal_i = df_jobs_anal_i.query('active_site == @var')

### Picking slab to test on

In [6]:
job_ids__completely_done__ads_o = df_jobs_anal_i.job_id_max
df_jobs_data_i = df_jobs_data.loc[job_ids__completely_done__ads_o]

# #########################################################
directory = os.path.join(
    os.environ["PROJ_irox_oer"],
    "dft_workflow/job_analysis/create_oh_slabs",
    "out_data/finished_O_ads")
if not os.path.exists(directory):
    os.makedirs(directory)

for job_id_i, row_data_i in df_jobs_data_i.iterrows():
    # #####################################################
    final_atoms_i = row_data_i.final_atoms
    # #####################################################
    file_name_i = job_id_i + ".cif"

    # final_atoms_i.write(os.path.join(directory, file_name_i))

    # final_atoms_i.write(os.path.join(
    #     "out_data",
    #     "finished_O_ads",
    #     file_name_i))

### Main Loop

In [7]:
df_jobs_anal_i = df_jobs_anal_i.droplevel(level=0)

data_dict_list = []
for name_i, row_i in df_jobs_anal_i.iterrows():

    if verbose:
        print(40 * "=")

    # #####################################################
    compenv_i = name_i[0]
    slab_id_i = name_i[1]
    ads_i = name_i[2]
    active_site_i = name_i[3]
    att_num_i = name_i[4]
    # #####################################################
    job_id_max_i = row_i.job_id_max
    # #####################################################

    if slab_id_i not in df_active_sites.index:
        print(244 * "slab_id not found in df_active_sites, need to run `get_all_active_sites.ipynb`")

    # #####################################################
    row_sites_i = df_active_sites.loc[slab_id_i]
    # #####################################################
    active_sites_unique_i = row_sites_i.active_sites_unique
    # #####################################################

    name_full_i = tuple(["oer_adsorbate", ] + list(name_i))

    # #####################################################
    # if name_i in df_atoms_sorted_ind.index:
    if name_full_i in df_atoms_sorted_ind.index:
        # row_atoms_i = df_atoms_sorted_ind.loc[name_i]
        row_atoms_i = df_atoms_sorted_ind.loc[name_full_i]
            # (compenv_i, slab_id_i, ads_i, active_site_i, att_num_i, )]
        # #####################################################
        atoms_sorted_good_i = row_atoms_i.atoms_sorted_good
        atoms = atoms_sorted_good_i
        # #####################################################




        # #####################################################
        name_i = (compenv_i, slab_id_i, ads_i, active_site_i, att_num_i, )


        # #####################################################
        # #####################################################
        for site_i in active_sites_unique_i:
            # print("site_i:", site_i)

            # #################################################
            # #################################################
            df_coord_i = get_df_coord(
                mode="post-dft",  # 'bulk', 'slab', 'post-dft'
                post_dft_name_tuple=name_i,
                )

            oh_slabs_list = get_ads_pos_oh(
                atoms=atoms,
                site_i=site_i,
                df_coord_i=df_coord_i,
                # #########################
                include_colinear=True,
                verbose=False,
                num_side_ads=3,
                )

            for att_num_oh_j, slab_oh_j in enumerate(oh_slabs_list):
                # #############################################
                data_dict_i = dict()
                # #############################################
                data_dict_i["compenv"] = compenv_i
                data_dict_i["slab_id"] = slab_id_i
                data_dict_i["ads"] = ads_i
                data_dict_i["active_site"] = site_i
                data_dict_i["att_num"] = att_num_i
                data_dict_i["att_num_oh"] = att_num_oh_j
                data_dict_i["slab_oh"] = slab_oh_j
                # #############################################
                data_dict_list.append(data_dict_i)
                # #############################################

df_slabs_oh = pd.DataFrame(data_dict_list)
df_slabs_oh = df_slabs_oh.set_index([
    "compenv", "slab_id", "ads",
    "active_site", "att_num", "att_num_oh", ])

# Writing *OH Slabs to File

In [8]:
directory = os.path.join(
    os.environ["PROJ_irox_oer"],
    "dft_workflow/job_analysis/create_oh_slabs",
    "out_data/oh_slabs")
if not os.path.exists(directory):
    os.makedirs(directory)

for name_i, row_i in df_slabs_oh.iterrows():
    # #####################################################
    slab_oh_i = row_i.slab_oh
    # #####################################################

    file_name_i = '__'.join(str(e) for e in list(name_i))
    file_name_i += ".cif"

    # if True:
    if False:
        slab_oh_i.write(os.path.join(directory, file_name_i))

# Save to pickle

In [9]:
# Pickling data ###########################################
import os; import pickle
directory = os.path.join(
    os.environ["PROJ_irox_oer"],
    "dft_workflow/job_analysis/create_oh_slabs",
    "out_data")
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "df_slabs_oh.pickle"), "wb") as fle:
    pickle.dump(df_slabs_oh, fle)
# #########################################################

In [10]:
from methods import get_df_slabs_oh

df_slabs_oh_tmp = get_df_slabs_oh()
# df_slabs_oh_tmp

In [11]:
# #########################################################
print(20 * "# # ")
print("All done!")
print("Run time:", np.round((time.time() - ti) / 60, 3), "min")
print("create_oh_slabs.ipynb")
print(20 * "# # ")
# #########################################################

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
All done!
Run time: 0.254 min
create_oh_slabs.ipynb
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
